In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, HfArgumentParser, TrainingArguments, pipeline, logging, TextStreamer
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
import os,torch, wandb, platform, warnings
import gradio 
from datasets import load_dataset
from trl import SFTTrainer

/home/dhkim/anaconda3/envs/llama_dhk/lib/python3.9/site-packages/torch/cuda/__init__.py:611: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
/home/dhkim/anaconda3/envs/llama_dhk/lib/python3.9/site-packages/trl/trainer/ppo_config.py:141: UserWarning: The `optimize_cuda_cache` arguement will be deprecated soon, please use `optimize_device_cache` instead.
  warnings.warn(


In [2]:
import torch
torch.cuda.empty_cache()

In [3]:
 print('메모리 사용량/총 메모리: ' + str(torch.cuda.memory_allocated('cuda:0')/1024**3 ) + 'GB / ' + str(torch.cuda.memory_reserved('cuda:0')/1024**3 ) + 'GB')

메모리 사용량/총 메모리: 0.0GB / 0.0GB


In [4]:
def print_system_specs():
    # Check if CUDA is available
    is_cuda_available = torch.cuda.is_available()
    print("CUDA Available:", is_cuda_available)
# Get the number of available CUDA devices
    num_cuda_devices = torch.cuda.device_count()
    print("Number of CUDA devices:", num_cuda_devices)
    if is_cuda_available:
        for i in range(num_cuda_devices):
            # Get CUDA device properties
            device = torch.device('cuda', i)
            print(f"--- CUDA Device {i} ---")
            print("Name:", torch.cuda.get_device_name(i))
            print("Compute Capability:", torch.cuda.get_device_capability(i))
            print("Total Memory:", torch.cuda.get_device_properties(i).total_memory, "bytes")
    # Get CPU information
    print("--- CPU Information ---")
    print("Processor:", platform.processor())
    print("System:", platform.system(), platform.release())
    print("Python Version:", platform.python_version())
print_system_specs()

CUDA Available: True
Number of CUDA devices: 1
--- CUDA Device 0 ---
Name: NVIDIA RTX A6000
Compute Capability: (8, 6)
Total Memory: 51041271808 bytes
--- CPU Information ---
Processor: x86_64
System: Linux 6.2.0-34-generic
Python Version: 3.9.12


In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("hyeogi/Yi-6b-dpo-v0.1")
model = AutoModelForCausalLM.from_pretrained("hyeogi/Yi-6b-dpo-v0.1")
model.to('cuda')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(78464, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=512, bias=False)
          (v_proj): Linear(in_features=4096, out_features=512, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm

In [6]:
# 예시 코드
example_sentence = "이것은 테스트 문장입니다."
tokenized_output = tokenizer.tokenize(example_sentence)
print(tokenized_output)

# 토크나이저와 모델의 임베딩 레이어 크기 비교
print(len(tokenizer.vocab))  # 토크나이저 단어장 크기

['▁이', '것은', '▁테스트', '▁문', '장', '입니다', '.']
78464


In [7]:
model.eval()

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(78464, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=512, bias=False)
          (v_proj): Linear(in_features=4096, out_features=512, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm

In [8]:
def stream(user_prompt):
    runtimeFlag = "cuda:0"
    system_prompt = "Below is an instruction that describes a task. Write a response that appropriately completes the request."
    B_INST, E_INST = "### Instruction:", "### Response:"

    prompt = f"{system_prompt}{B_INST}{user_prompt.strip()}{E_INST}"

    inputs = tokenizer([prompt], return_tensors="pt").to(runtimeFlag)

    # Generate response
    outputs = model.generate(**inputs, max_new_tokens=1000, temperature=1.0, length_penalty=0.2, do_sample=True, num_beams=10)

    # Decode and print the generated response
    print(tokenizer.decode(outputs[0]))
    
    del outputs, inputs
    # Empty the cuda cache
    torch.cuda.empty_cache()


stream("나 오늘 슬퍼")

<|startoftext|> Below is an instruction that describes a task. Write a response that appropriately completes the request.### Instruction:나 오늘 슬퍼### Response: ..........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [9]:
def stream(user_prompt):
    runtimeFlag = "cuda:0"
    system_prompt = "As a world-class psychologist, I need your assistance in providing comfort to individuals who are seeking emotional support. Please act as a compassionate and friendly psychologist and offer guidance and reassurance to those in need. Here are some topics to include in your response."
    B_INST, E_INST = "### Instruction:", "### Response:"

    prompt = f"{system_prompt}{B_INST}{user_prompt.strip()}{E_INST}"

    inputs = tokenizer([prompt], return_tensors="pt").to(runtimeFlag)

    # Generate response
    outputs = model.generate(**inputs, max_new_tokens=1000, temperature=1.0, length_penalty=0.2, do_sample=True, num_beams=10)

    # Decode and print the generated response
    print(tokenizer.decode(outputs[0]))
    
    del outputs, inputs
    # Empty the cuda cache
    torch.cuda.empty_cache()

stream("나 오늘 슬퍼")


<|startoftext|> As a world-class psychologist, I need your assistance in providing comfort to individuals who are seeking emotional support. Please act as a compassionate and friendly psychologist and offer guidance and reassurance to those in need. Here are some topics to include in your response.### Instruction:나 오늘 슬퍼### Response: ????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????

In [10]:
def stream(user_prompt):
    runtimeFlag = "cuda:0"
    system_prompt = "As a world-class psychologist, I need your assistance in providing comfort to individuals who are seeking emotional support. Please act as a compassionate and friendly psychologist and offer guidance and reassurance to those in need. Here are some topics to include in your response."
    B_INST, E_INST = "### Instruction:", "### Response:"

    prompt = f"{system_prompt}{B_INST}{user_prompt.strip()}{E_INST}"

    inputs = tokenizer([prompt], return_tensors="pt").to(runtimeFlag)

    # Generate response
    outputs = model.generate(**inputs, max_new_tokens=500, temperature=1.0, length_penalty=0.2, do_sample=True, num_beams=5)

    # Decode and print the generated response
    print(tokenizer.decode(outputs[0]))
    
    del outputs, inputs
    # Empty the cuda cache
    torch.cuda.empty_cache()

stream("나 오늘 떡볶이 먹어서 기뻐")



<|startoftext|> As a world-class psychologist, I need your assistance in providing comfort to individuals who are seeking emotional support. Please act as a compassionate and friendly psychologist and offer guidance and reassurance to those in need. Here are some topics to include in your response.### Instruction:나 오늘 떡볶이 먹어서 기뻐### Response: ???????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????
